# Análise de dados

PySpark é a interface alto nível que permite você conseguir acessar e usar o Spark por meio da linguagem Python. Usando o PySpark, você consegue escrever todo o seu código usando apenas o nosso estilo Python de escrever código.

## Estrutura do notebook

[1. Importação das bibliotecas](#1.-Importa%C3%A7%C3%A3o-das-bibliotecas)<br>
[2. Informações da amostra](#2.-Informa%C3%A7%C3%B5es-da-base)<br>
[3. Escolhendo e tratando colunas](#3.-Escolhendo-e-tratando-colunas-desejadas)<br>
[4. Modelo de regressão](#4.-Obtendo-informa%C3%A7%C3%B5es-atraves-de-graficos)

### 1. Importação das bibliotecas

In [1]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

### 2. Informações da amostra

In [2]:
#Leitura da base
df = spark.read.csv('/home/tatiane/Downloads/Consumo_cerveja.csv', sep = ',', header = True)

In [3]:
df.limit(10).toPandas()

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
0,2015-01-01,"27,3","23,9","32,5",0,0,25.461
1,2015-01-02,"27,02","24,5","33,5",0,0,28.972
2,2015-01-03,"24,82","22,4","29,9",0,1,30.814
3,2015-01-04,"23,98","21,5","28,6","1,2",1,29.799
4,2015-01-05,"23,82",21,"28,3",0,0,28.900
5,2015-01-06,"23,78","20,1","30,5","12,2",0,28.218
6,2015-01-07,24,"19,5","33,7",0,0,29.732
7,2015-01-08,"24,9","19,5","32,8","48,6",0,28.397
8,2015-01-09,"28,2","21,9",34,"4,4",0,24.886
9,2015-01-10,"26,76","22,1","34,2",0,1,37.937


In [4]:
#Quantidade de registros
print('Quantidade de registros:', df.count())
print('Quantidade de colunas:', len(df.columns))

Quantidade de registros: 941
Quantidade de colunas: 7


### 3. Escolhendo e tratando colunas desejadas

É essencial escolher as colunas que vai precisar, aqui a base é pequena então até seria tranquilo trabalhar com todas, mas não se pode dizer o mesmo com bases de 40 milhoes de registros.
O objetivo dessa análise é (try) descobrir o mês em que mais se consome cerveja.

In [5]:
#criando colunas dia, mês e ano, extraindo-as da coluna Data.
df = df.withColumn('dia', F.dayofmonth('Data'))
df = df.withColumn('mes', F.month('Data'))
df = df.withColumn('ano', F.year('Data'))

In [6]:
df.select('Data', 'ano', 'mes', 'dia').limit(200).toPandas()

,Data,ano,mes,dia
0,2015-01-01,2015,1,1
1,2015-01-02,2015,1,2
2,2015-01-03,2015,1,3
3,2015-01-04,2015,1,4
4,2015-01-05,2015,1,5
...,...,...,...,...
195,2015-07-15,2015,7,15
196,2015-07-16,2015,7,16
197,2015-07-17,2015,7,17
198,2015-07-18,2015,7,18


In [7]:
df.printSchema()

root
 |-- Data: string (nullable = true)
 |-- Temperatura Media (C): string (nullable = true)
 |-- Temperatura Minima (C): string (nullable = true)
 |-- Temperatura Maxima (C): string (nullable = true)
 |-- Precipitacao (mm): string (nullable = true)
 |-- Final de Semana: string (nullable = true)
 |-- Consumo de cerveja (litros): string (nullable = true)
 |-- dia: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [8]:
#renomeando colunas
df = df.withColumnRenamed("Temperatura Media (C)","temp_mediaC")
df = df.withColumnRenamed("Temperatura Minima (C)","temp_minimaC")
df = df.withColumnRenamed("Temperatura Maxima (C)","temp_maximaC")
df = df.withColumnRenamed("Precipitacao (mm)","precipitacao_mm")
df = df.withColumnRenamed("Final de Semana","final_semana")
df = df.withColumnRenamed("Consumo de cerveja (litros)","consumo_litros")

In [9]:
df.limit(10).toPandas()

,Data,temp_mediaC,temp_minimaC,temp_maximaC,precipitacao_mm,final_semana,consumo_litros,dia,mes,ano
0,2015-01-01,"27,3","23,9","32,5",0,0,25.461,1,1,2015
1,2015-01-02,"27,02","24,5","33,5",0,0,28.972,2,1,2015
2,2015-01-03,"24,82","22,4","29,9",0,1,30.814,3,1,2015
3,2015-01-04,"23,98","21,5","28,6","1,2",1,29.799,4,1,2015
4,2015-01-05,"23,82",21,"28,3",0,0,28.900,5,1,2015
5,2015-01-06,"23,78","20,1","30,5","12,2",0,28.218,6,1,2015
6,2015-01-07,24,"19,5","33,7",0,0,29.732,7,1,2015
7,2015-01-08,"24,9","19,5","32,8","48,6",0,28.397,8,1,2015
8,2015-01-09,"28,2","21,9",34,"4,4",0,24.886,9,1,2015
9,2015-01-10,"26,76","22,1","34,2",0,1,37.937,10,1,2015


In [10]:
for i in df.columns:
    print(i)
    nao_nulos = df.filter(F.col(i).isNotNull()).count()
    print('Número de preenchidos: ', nao_nulos)
    nulos = df.filter(F.col(i).isNull()).count()
    print('Número de nulos: ', nulos)
    unicos = df.select(i).distinct().count()
    print('Número de únicos: ', unicos)
    unicos_nao_nulos = df.dropDuplicates([i]).filter(F.col(i).isNotNull()).count()
    print('Número de únicos não nulos: ', unicos_nao_nulos)
    print('\n')

Data
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  366
Número de únicos não nulos:  365


temp_mediaC
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  278
Número de únicos não nulos:  277


temp_minimaC
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  111
Número de únicos não nulos:  110


temp_maximaC
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  152
Número de únicos não nulos:  151


precipitacao_mm
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  94
Número de únicos não nulos:  93


final_semana
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  3
Número de únicos não nulos:  2


consumo_litros
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  360
Número de únicos não nulos:  359


dia
Número de preenchidos:  365
Número de nulos:  576
Número de únicos:  32
Número de únicos não nulos:  31


mes
Número de preenchidos:  365
Número de nulos:  576

In [11]:
#apagando valores nulos para nao interferir
df = df.dropna()

In [13]:
nulos = df.filter(F.col('consumo_litros').isNull()).count()
print('Número de nulos: ', nulos)

Número de nulos:  0


In [16]:
# Converter o tipo float
df = df.withColumn('consumo_litros', F.col('consumo_litros').cast(FloatType()))
df = df.withColumn('Data', F.col('data').cast(DateType()))

In [26]:
df.printSchema()

root
 |-- Data: date (nullable = true)
 |-- temp_mediaC: string (nullable = true)
 |-- temp_minimaC: string (nullable = true)
 |-- temp_maximaC: string (nullable = true)
 |-- precipitacao_mm: string (nullable = true)
 |-- final_semana: string (nullable = true)
 |-- consumo_litros: float (nullable = true)
 |-- dia: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [27]:
df.summary().toPandas()

,summary,temp_mediaC,temp_minimaC,temp_maximaC,precipitacao_mm,final_semana,consumo_litros,dia,mes,ano
0,count,365,365,365,365,365,365,365,365,365
1,mean,23.22222222222222,16.322580645161292,27.341463414634145,1.5,0.28493150684931506,25.401367111728614,15.72054794520548,6.526027397260274,2015.0
2,stddev,3.80058475033046,3.145230217055591,4.424984497699178,7.827099429365802,0.4520014200833207,4.399142676215856,8.808321472985572,3.452584133232603,0.0
3,min,"12,9","10,6","14,5",0,0,14.343,1,1,2015
4,25%,24.0,14.0,25.0,0.0,0.0,22.008,8,4,2015
5,50%,24.0,17.0,28.0,0.0,0.0,24.867,16,7,2015
6,75%,24.0,19.0,30.0,0.0,1.0,28.631,23,10,2015
7,max,"28,86","24,5","36,5","94,8",1,37.937,31,12,2015


### 4. Modelo de regressão

In [29]:
conda install -c https://conda.anaconda.org/plotly plotly

Solving environment: done

## Package Plan ##

  environment location: /home/tatiane/miniconda3

  added / updated specs:
    - plotly


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.10.11 |       h06a4308_0         124 KB
    certifi-2022.9.24          |   py39h06a4308_0         154 KB
    conda-22.11.0              |   py39h06a4308_1         927 KB
    openssl-1.1.1s             |       h7f8727e_0         3.6 MB
    plotly-5.11.0              |             py_0         7.0 MB  plotly
    pluggy-1.0.0               |   py39h06a4308_1          28 KB
    ruamel.yaml-0.16.12        |   py39h27cfd23_1         184 KB
    ruamel.yaml.clib-0.2.6     |   py39h7f8727e_0         137 KB
    tenacity-8.0.1             |   py39h06a4308_1          34 KB
    toolz-0.12.0               |   py39h06a4308_0         105 KB
    ---------------------------------------------------------

In [30]:
# Visualização
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

# Regressão Linear
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Erros
import warnings
warnings.filterwarnings('ignore')


In [31]:
df.printSchema()

root
 |-- Data: date (nullable = true)
 |-- temp_mediaC: string (nullable = true)
 |-- temp_minimaC: string (nullable = true)
 |-- temp_maximaC: string (nullable = true)
 |-- precipitacao_mm: string (nullable = true)
 |-- final_semana: string (nullable = true)
 |-- consumo_litros: float (nullable = true)
 |-- dia: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
#consumo de cerveja durante o ano

fig, ax = plt.subplots(figsize=(15, 8))

ax = df.filter('consumo_litros').plot()
ax.set_title('Consumo de cerveja X ANO')
ax.set_ylabel("Litros de cerveja em dezenas de milhares", fontsize= 15)
ax.set_xlabel('Dias durante o ano', fontsize= 15);